# IPython Parallel 

In regular IPython we have a client (the frontend) and a kernel which executes the code. And they communciate with messages. 

So, as IPython already does remote execution... if you have _one_ remote kernel, why not have _one hundred_?

These are called IPython Parallel "engines"

<div>
<img src="ipyparallel.png" style="width:300px"/>
</div>
Rather than having clients (blue) connect directly to kernels (green) as in notebook, you have an intermediary of a hub (with schedulers) - known as the "controller". The client communicates only with the controller. The controller keeps track of the available engines and forwards requests from the client to the engines. It schedules the work and monitors its status. The results are communicated through the controller back to the client.

To use IPython for parallel computing, you need to start one instance of the controller and one or more instances of the engines. The controller and each engine can run on different machines or on the same machine.

There are two ways of going about starting a controller and engines:

- Separately, using the **ipcontroller** and **ipengine** commands.
- In an automated manner using the **ipcluster** command.


<div class="alert alert-block alert-info">
    <b>Note:</b> The following commands need to be entered in a terminal. File > New > Terminal. A terminal will open as a new tab. Grab the tab and pull it to the right to have the terminal next to your notebook. The terminal does not have the same environment loaded as the notebook. To fix that we need to "source pythonhpc". 
</div>

```
$ source pythonhpc.sh
$ ipcluster start --n 4  &
```    

Now let's see how we access the "Cluster". [IPython][IP] comes with a module [ipyparallel][IPp] that is used to access the engines, we just started. We first need to import Client.

[IPp]: https://ipyparallel.readthedocs.io/en/latest/
[IP]: http://www.ipython.org

The client is started by first importing it from ipyparallel and then by initalizing it. 

In [ ]:
import ipyparallel as ipp

<div class="alert alert-block alert-danger">
    <b>Note:</b> If you receive an error ModuleNotFoundError: No module named 'ipyparallel', ensure that you have the miniconda kernel loaded
</div>


In [ ]:
rc = ipp.Client(profile="default")

You can set up "profiles" - you could have a profile for your laptop, one that connects you to a remote cluster, and so on. The above connects the client through the controller to the engines. "rc" will now contain a list of engines, which we can list. 

List the ids of the engines attached:

In [ ]:
rc.ids

## Views
Generally we don't act on these engines directly. Instead we create a view first. 

Views gives us access to a set of engines using a given scheduler. 

There are two types of views

- direct view
- load balanced view

A *direct view* gives us direct control of the engines.  We can push and pull data and apply functions using a couple of different methods. We are in control what runs where.

A *load-balanced view* tries to balance the work between the engines. We can submit tasks to it in the same way as before, but with a *load-balanced view*, the scheduler decides where a function is executed.

You can create DirectViews to single engines simply by accessing the client by engine id:

In [ ]:
rc[0]

You can also create a DirectView with a list of engines:

In [ ]:
rc[0,1,2,3]

Or you can slice and dice:

In [ ]:
#v01 = rc[0:2] #engines 0,1
#v23 = rc[2:4] #engines 2,3
#even = rc[::2] #stride of 2
#odd = rc[1::2] #stride of 2, offset of 1
dview = rc[:] # all available engines

The load balanced view always runs tasks on exactly one engine, but it let the scheduler decide where that should be. Load-balanced views can be created with the client’s view method:


In [ ]:
lbv = rc.load_balanced_view()

## Using the Direct View

As mentioned above a *direct view* lets you control each engine directly. You can also decide if a command should be blocking or not.

In [ ]:
rc.block = True # we want blocking calls for now

In [ ]:
def mul(a, b):
    return a*b

In [ ]:
def summary():
    """ info about this process"""
    import os
    import socket
    import sys
    return {
        'cwd': os.getcwd(),
        'Python': sys.version,
        'hostname': socket.gethostname(),
        'pid': os.getpid(),
    }

In [ ]:
mul(5, 6)

In [ ]:
summary()

### Remote execution: view.apply

The basic API for remote execution with IPython is `view.apply`.

Create a view, and instead of calling the function locally, 
pass the function and its arguments. So instead of mul(5, 6) call apply(mul, 5 ,6)

In [ ]:
rc[0].apply(mul, 5 ,6)

In [ ]:
rc[0].apply(summary)

Although the hostnames are the same (we are running all processes on a single node) the pids are different. Summary function was called on the remote engine. 
IPython packed up the function, packed up the arguments, sent them over to engine and it called it and sent the return back.


In [ ]:
d = _
d['pid']

And in parallel:

In [ ]:
rc[:].apply(mul, 5, 6)

You get a list which is the same shape as the engines.

In [ ]:
rc[:].apply(summary)

Python has a built-in function for mapping. You call map with a function and iterators of arguments.

In [ ]:
list(map(mul, range(1,10), range(2,11)))

In parallel we have the map method.

In [ ]:
list(dview.map(mul, range(1,10), range(2,11)))

## Moving objects around 

You can transfer Python objects to and from the engines. In IPython, these operations are called `push()` (sending an object to the engines) and `pull()` (getting an object from the engines).

Here are some examples

In [ ]:
dview.block = True
dview.push(dict(a=1.032, b=3453))

In [ ]:
dview.pull('a')

In [ ]:
dview.pull('b', targets=0)

In [ ]:
dview.pull(('a', 'b'))

Note: if you are using non-blocking then `push()` and `pull()` also return `AsyncResult` objects:

In [ ]:
ar = dview.pull('a', block=False)
#ar

In [ ]:
ar.get()

## Scatter and gather

It might be useful to partition a sequence and push the partitions to different engines. In MPI, this is known as scatter/gather. Here, however, scatter() is from the interactive IPython/Notebook to the engines and gather() is from the engines back to the interactive IPython/Notebook. For scatter/gather operations between engines we can use MPI.

In [ ]:
dview.scatter('a',range(16))

In [ ]:
dview['a']

In [ ]:
dview.gather('a') # This will show you the status of gather.

In [ ]:
#you can also do async
dview.block = False
dview.scatter('a',range(32))
dview.gather('a').get() # This will give you the result.

### Remote function decorators

These are like normal functions but when called they execute on one or more engines instead than locally. IPython provides two decorators for producing parallel functions:

The first is `@remote`, which calls the function on all engines in a view.

In [ ]:
@dview.remote(block=True)
def getpid():
    import os
    return os.getpid()

In [ ]:
getpid()

The second is `@parallel`. It creates parallel functions that break up element-wise operations and distribute them, reconstructing the result.

In [ ]:
import numpy as np
A = np.random.random((64,48))
@dview.parallel(block=True)
def pmul(A,B):
    return A*B

C_local = A*A
C_remote = pmul(A,A)

(C_local == C_remote).all()

## Parallel Magics

IPython makes it very easy to use IPyParallel. It provides the magic commands ``%px`` and ``%%px`` to execute code in parallel. The target attribute is used to pick the engines, you want. By default, all the engines of the last Client object created are used. You can also specify if a command should be executed `blocking` or `non-blocking`.

Note, the commands prefixed with ``%px`` are not executed locally!

In [ ]:
%px import numpy as np # import numpy on all engines as np
import numpy as np # do it locally, too.

Since it's fairly common that you want to execute a cell remotely and locally, there's an option for that. Just add ``--local``.

**Note**: This works only for ``%%px`` not ``%px``.

In [ ]:
%%px --local 
np.__version__ # print the numpy version of the engines. Note how the output is prefixed. It can be accessed that way, too. 

 The engines run IPython. Magic commands work, too.

In [ ]:
%%px --local
%matplotlib inline

In [ ]:
%%px --local 
import matplotlib.pyplot as plt


The cell magic command ``%%px`` lets us execute more than one statement. The option ``--target`` lets us choose which engines we want to use. Here we are using engines 0 to 1.

In [ ]:
%%px --target 0:2
a = np.random.random([10,10])
plt.imshow(a, interpolation="nearest")

Remember that the imports we do with ``%px`` are not available directly in our notebook. We can fix that by using:

In [ ]:
with rc[:].sync_imports():
    import matplotlib.pyplot

In [ ]:
%%px 
import os, socket
print(os.getpid())
print(socket.gethostname())

You can say you want something executed both locally and on the engines with an option to the magic:

In [ ]:
%%px --local
import numpy

The above is equivalent to:

In [ ]:
%px import numpy as np
import numpy as np

In [ ]:
%%px --local
print(np.__version__, np.__file__)